# Pay to win: generate loads of art in record time using the openai image generation API

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## install libs



In [1]:
# --- 1) Load key from colab secrets (must be set in "Secrets" panel) ---
import os
from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# --- 2) Install & import client ---
!pip install --quiet openai python-slugify

## define image generation and saving function

In [8]:
# --- Colab cell: FLUX.1-schnell via Together AI with URL fallback + logging ---
import os
import base64
import json
import requests
from datetime import datetime
from slugify import slugify

# Where to save images & log
SAVE_DIR = "/content/drive/MyDrive/AI/together_flux"
LOG_PATH = os.path.join(SAVE_DIR, "together_flux.log")
os.makedirs(SAVE_DIR, exist_ok=True)

TOGETHER_API_URL = "https://api.together.xyz/v1/images/generations"
MODEL_NAME = "black-forest-labs/FLUX.1-schnell"  # Together's FLUX schnell model

def _get_together_key() -> str:
    key = os.environ.get("TOGETHER_AI")
    if key:
        return key
    try:
        from google.colab import userdata
        key = userdata.get("TOGETHER_AI")
    except Exception:
        key = None
    if not key:
        raise RuntimeError("Missing Together API key. Set Colab secret or env var named TOGETHER_AI.")
    return key

def _slug_first_last(text: str, n: int = 50) -> str:
    text = (text or "").strip()
    if len(text) <= 2 * n:
        snippet = text
    else:
        snippet = text[:n] + " " + text[-n:]
    return slugify(snippet) or "image"

def _append_log(timestamp: str, filename: str, url: str, prompt: str) -> None:
    # CSV-like .log: timestamp,filename,url,prompt (prompt last; may contain commas)
    with open(LOG_PATH, "a", encoding="utf-8") as logf:
        logf.write(f"{timestamp},{filename},{url},{prompt}\n")

def save_image_from_prompt(
    prompt: str,
    width: int = 1024,
    height: int = 1024,
    model: str = MODEL_NAME,
) -> str:
    """
    Generate an image from a prompt using Together AI (FLUX.1-schnell).
    Handles both base64 and URL responses.
    Logs (timestamp, filename, url, prompt) to together_flux.log.
    Saves image under /content/drive/MyDrive/AI/together_flux and returns the file path.
    """
    api_key = _get_together_key()

    payload = {
        "model": model,
        "prompt": prompt,
        "width": width,
        "height": height,
        "n": 1,
        # Optional params you can enable:
        # "seed": 0, "steps": 20, "guidance": 3.5
    }

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    resp = requests.post(TOGETHER_API_URL, headers=headers, data=json.dumps(payload), timeout=180)
    if resp.status_code != 200:
        raise RuntimeError(f"Together API error {resp.status_code}: {resp.text[:500]}")

    data = resp.json()
    if "data" not in data or not data["data"]:
        raise RuntimeError(f"Unexpected Together API response: {data}")

    # Filename
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    slug = _slug_first_last(prompt, 50)
    fname = f"{timestamp}-{slug}.png"
    fpath = os.path.join(SAVE_DIR, fname)

    # Prefer base64 if present, else use URL
    entry = data["data"][0]
    b64 = entry.get("b64_json")
    url = entry.get("url")

    if b64:
        image_bytes = base64.b64decode(b64)
        with open(fpath, "wb") as f:
            f.write(image_bytes)
        if url:
            _append_log(timestamp, fname, url, prompt)  # log URL if provided alongside b64
    elif url:
        # Log first, then download
        _append_log(timestamp, fname, url, prompt)
        # Some Together short links are public; if authorization is needed, try header too.
        get_headers = {}
        try:
            # Try without auth header first
            r = requests.get(url, timeout=300)
            if r.status_code == 401 or r.status_code == 403:
                get_headers = {"Authorization": f"Bearer {api_key}"}
                r = requests.get(url, headers=get_headers, timeout=300)
        except requests.RequestException as e:
            raise RuntimeError(f"Failed to download image from URL: {e}")

        if r.status_code != 200:
            raise RuntimeError(f"Failed to download image from URL (status {r.status_code}): {r.text[:300]}")
        with open(fpath, "wb") as f:
            f.write(r.content)
    else:
        # Neither b64 nor URL present
        raise RuntimeError(f"No image data returned: {data}")

    print(f"Saved → {fpath}")
    return fpath


Example run (deactivated to go faster)

In [9]:
# save_image_from_prompt('a delicious pizza with large zucchini flowers and blue cheese')

Saved → /content/drive/MyDrive/AI/together_flux/20251019-104338-a-delicious-pizza-with-large-zucchini-flowers-and-blue-cheese.png


'/content/drive/MyDrive/AI/together_flux/20251019-104338-a-delicious-pizza-with-large-zucchini-flowers-and-blue-cheese.png'

## Look at images generated by openai dalle3

There have been failures in the openai generation pipeline, so we will only generate images for the successes.  

In [12]:
import pandas as pd


df = pd.read_csv('/content/drive/MyDrive/AI/dalle3/generated_prompts.csv')

We see that over 232 prompts, we have 46 failures, and 186 successes.  

In [18]:
len(df[df.file.isna()]) ,len(df), len(df[~df.file.isna()])

(46, 232, 186)

In [19]:
df = df[~df.file.isna()]

In [20]:
df.head()

,prompt,file,error
0,"impressive oil painting, intimate portrait of ...",/content/drive/MyDrive/AI/dalle3/20251019-0038...,NaN
2,"powerful oil painting, portrayal of a strong f...",/content/drive/MyDrive/AI/dalle3/20251019-0040...,NaN
3,"intriguing oil painting, enigmatic portrait of...",/content/drive/MyDrive/AI/dalle3/20251019-0041...,NaN
4,"vibrant oil painting, lively interior scene wi...",/content/drive/MyDrive/AI/dalle3/20251019-0041...,NaN
5,"striking woodblock print, dramatic seascape wi...",/content/drive/MyDrive/AI/dalle3/20251019-0042...,NaN


## Run the batch

In [ ]:
import os
import time
import pandas as pd
from tqdm.auto import tqdm

out_dir = "/content/drive/MyDrive/AI/together_flux"
os.makedirs(out_dir, exist_ok=True)

log_path = os.path.join(out_dir, "generated_prompts.csv")

# --- Initialize CSV file with header once if not exists
if not os.path.exists(log_path):
    pd.DataFrame(columns=["prompt", "file", "error"]).to_csv(log_path, index=False)

for row in tqdm(df.itertuples(), total=len(df), desc="Generating"):
    prompt = row.prompt
    try:
        fname = save_image_from_prompt(prompt)
        err = None
    except Exception as e:
        fname = None
        err = str(e)

    # Append immediately
    pd.DataFrame([{
        "prompt": prompt,
        "file": fname,
        "error": err
    }]).to_csv(log_path, mode="a", index=False, header=False)

    # Sleep also after errors
    time.sleep(11)


Generating:   0%|          | 0/186 [00:00<?, ?it/s]

Saved → /content/drive/MyDrive/AI/together_flux/20251019-105250-impressive-oil-painting-intimate-portrait-of-an-a-dutch-golden-age-emotionally-resonant-masterpiece.png
Saved → /content/drive/MyDrive/AI/together_flux/20251019-105303-powerful-oil-painting-portrayal-of-a-strong-femal-ntileschi-meets-baroque-art-inspiring-canvas-work.png
Saved → /content/drive/MyDrive/AI/together_flux/20251019-105316-intriguing-oil-painting-enigmatic-portrait-of-a-n-s-the-italian-renaissance-captivating-masterpiece.png
Saved → /content/drive/MyDrive/AI/together_flux/20251019-105329-vibrant-oil-painting-lively-interior-scene-with-b-enri-matisse-meets-fauvism-enchanting-canvas-work.png
Saved → /content/drive/MyDrive/AI/together_flux/20251019-105341-striking-woodblock-print-dramatic-seascape-with-c-ushika-hokusai-meets-ukiyo-e-breathtaking-artwork.png
Saved → /content/drive/MyDrive/AI/together_flux/20251019-105354-sensual-oil-painting-close-up-of-a-blooming-flowe-ffe-meets-american-modernism-alluring-canvas-w